In [15]:
import numpy as np
import pandas as pd
import os

In [16]:
data_dir = 'vehicles.csv'

In [17]:
data = pd.read_csv( data_dir)


### Preprocessing

In [19]:
data

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,...,NaN,sedan,NaN,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,...,NaN,sedan,red,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,...,NaN,hatchback,white,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,NaN,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,...,NaN,sedan,silver,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


In [20]:
data.isna()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,False,False,False,False,False,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
1,False,False,False,False,False,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
2,False,False,False,False,False,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
3,False,False,False,False,False,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
4,False,False,False,False,False,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,False,False,False,False,False,False,False,False,False,False,...,True,False,True,False,False,True,False,False,False,False
426876,False,False,False,False,False,False,False,False,False,True,...,True,False,False,False,False,True,False,False,False,False
426877,False,False,False,False,False,False,False,False,False,True,...,True,False,False,False,False,True,False,False,False,False
426878,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False


In [21]:
data.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
image_url           68
description         70
county          426880
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

In [22]:
data.isna().mean()

id              0.000000
url             0.000000
region          0.000000
region_url      0.000000
price           0.000000
year            0.002823
manufacturer    0.041337
model           0.012362
condition       0.407852
cylinders       0.416225
fuel            0.007058
odometer        0.010307
title_status    0.019308
transmission    0.005988
VIN             0.377254
drive           0.305863
size            0.717675
type            0.217527
paint_color     0.305011
image_url       0.000159
description     0.000164
county          1.000000
state           0.000000
lat             0.015342
long            0.015342
posting_date    0.000159
dtype: float64

In [23]:
data_v1 = data.drop(data.columns[data.isna().mean() > 0.25], axis = 1)

data_v1

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,type,image_url,description,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,gas,32226.0,clean,other,sedan,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,gas,12029.0,clean,other,sedan,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,diesel,4174.0,clean,other,hatchback,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,gas,30112.0,clean,other,sedan,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


In [107]:
unneeded_columns = ['id', 'url', 'region_url', 'image_url', 'description', 'posting_date', 'model']

data_v2 = data_v1.drop(unneeded_columns, axis = 1)

data_v2

,region,price,year,manufacturer,fuel,odometer,title_status,transmission,type,state,lat,long
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,nissan,gas,32226.0,clean,other,sedan,wy,33.786500,-84.445400
426876,wyoming,30590,2020.0,volvo,gas,12029.0,clean,other,sedan,wy,33.786500,-84.445400
426877,wyoming,34990,2020.0,cadillac,diesel,4174.0,clean,other,hatchback,wy,33.779214,-84.411811
426878,wyoming,28990,2018.0,lexus,gas,30112.0,clean,other,sedan,wy,33.786500,-84.445400


In [108]:
data_v2

,region,price,year,manufacturer,fuel,odometer,title_status,transmission,type,state,lat,long
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,nissan,gas,32226.0,clean,other,sedan,wy,33.786500,-84.445400
426876,wyoming,30590,2020.0,volvo,gas,12029.0,clean,other,sedan,wy,33.786500,-84.445400
426877,wyoming,34990,2020.0,cadillac,diesel,4174.0,clean,other,hatchback,wy,33.779214,-84.411811
426878,wyoming,28990,2018.0,lexus,gas,30112.0,clean,other,sedan,wy,33.786500,-84.445400


In [27]:
{column: len(data[column].unique()) for column in data_v2.columns if data_v2.dtypes[column] == 'object'}

{'region': 404,
 'manufacturer': 43,
 'fuel': 6,
 'title_status': 7,
 'transmission': 4,
 'type': 14,
 'state': 51}

In [19]:
pd_d

,price,year,odometer,lat,long
0,6000.0,2011.235191,98043.331443,38.493940,-94.748599
1,11900.0,2011.235191,98043.331443,38.493940,-94.748599
2,21000.0,2011.235191,98043.331443,38.493940,-94.748599
3,1500.0,2011.235191,98043.331443,38.493940,-94.748599
4,4900.0,2011.235191,98043.331443,38.493940,-94.748599
...,...,...,...,...,...
426875,23590.0,2019.000000,32226.000000,33.786500,-84.445400
426876,30590.0,2020.000000,12029.000000,33.786500,-84.445400
426877,34990.0,2020.000000,4174.000000,33.779214,-84.411811
426878,28990.0,2018.000000,30112.000000,33.786500,-84.445400


In [27]:
def onehot_encode(df, columns, prefixes):
  df = df.copy()
  for column, prefix in zip(columns, prefixes):
    dummies = pd.get_dummies(df[column], prefix = prefix)
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(column, axis = 1)
  return df

In [28]:
data_v2_1h = onehot_encode(data_v2, ['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'type', 'state'], ['reg', 'manu', 'fuel', 'title', 'trans', 'type', 'state'])

In [29]:
data_v2_1h

,price,year,odometer,lat,long,reg_SF bay area,reg_abilene,reg_akron / canton,reg_albany,reg_albuquerque,reg_altoona-johnstown,reg_amarillo,reg_ames,reg_anchorage / mat-su,reg_ann arbor,reg_annapolis,reg_appleton-oshkosh-FDL,reg_asheville,reg_ashtabula,reg_athens,reg_atlanta,reg_auburn,reg_augusta,reg_austin,reg_bakersfield,reg_baltimore,reg_baton rouge,reg_battle creek,reg_beaumont / port arthur,reg_bellingham,reg_bemidji,reg_bend,reg_billings,reg_binghamton,reg_birmingham,reg_bismarck,reg_bloomington,reg_bloomington-normal,reg_boise,reg_boone,...,state_hi,state_ia,state_id,state_il,state_in,state_ks,state_ky,state_la,state_ma,state_md,state_me,state_mi,state_mn,state_mo,state_ms,state_mt,state_nc,state_nd,state_ne,state_nh,state_nj,state_nm,state_nv,state_ny,state_oh,state_ok,state_or,state_pa,state_ri,state_sc,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
0,6000,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,11900,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,21000,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1500,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4900,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,23590,2019.0,32226.0,33.786500,-84.445400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
426876,30590,2020.0,12029.0,33.786500,-84.445400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
426877,34990,2020.0,4174.0,33.779214,-84.411811,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
426878,28990,2018.0,30112.0,33.786500,-84.445400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [31]:
#Removing NaNs
for column in data_v2_1h.columns:
  data_v2_1h[column] = data_v2_1h[column].fillna(data_v2_1h[column].mean())

In [32]:
data_v2_1h.isna().sum().sum()

0

### Splitting and Scaling

In [33]:
y = data_v2_1h.loc[:, 'price']
X = data_v2_1h.drop('price', axis=1)

In [34]:
scaler = StandardScaler()

In [35]:
X = scaler.fit_transform(X)

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 42)

### Training

In [38]:
from sklearn.linear_model import LinearRegression

In [39]:
linear_model = LinearRegression()

In [41]:
linear_model.fit(X_train, y_train)

lin_y_preds = linear_model.predict(X_test)


### Using lightGBM for gradient-boosting

In [43]:
import lightgbm as lgb  

In [44]:
lgb_model = lgb.LGBMRegressor(
    boosting_type = 'gbdt',
    reg_lambda = 1.0
)
lgb_model.fit(X_train, y_train)

lgb_y_preds = lgb_model.predict(X_test)

In [47]:
from sklearn.metrics import mean_squared_error

In [48]:
lin_loss = np.sqrt(mean_squared_error(y_test, lin_y_preds))
lgb_loss = np.sqrt(mean_squared_error(y_test, lgb_y_preds))

In [49]:
print('Linear Regression RMSE: ', lin_loss)
print('LGB RMSE: ', lgb_loss)

Linear Regression RMSE:  1.0742660579917032e+17
LGB RMSE:  16263542.54531327


In [51]:
print('Linear Regression R^2 Score: ', linear_model.score(X_test, y_test))
print('GBM R^2 Score: ', lgb_model.score(X_test, y_test))

#Can be better!

Linear Regression R^2 Score:  -4.4219872234120225e+19
GBM R^2 Score:  -0.013500752131093918



### Data set modifications using a pipeline

In [30]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin

In [31]:
# impute = SimpleImputer(strategy = 'mean')
# data_v2_num = data_v2.drop(['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'type', 'state'], axis = 1)

In [39]:
# encoder = OneHotEncoder()

In [69]:
# class DataFrameSelector(BaseEstimator, TransformerMixin):
#   def __init__(self, attribute_names):
#     self.attribute_names = attribute_names
#   def fit(self, X, y=None):
#     return self
#   def transform(self, X):
#     return X[self.attribute_names].values

In [109]:
data_v2

,region,price,year,manufacturer,fuel,odometer,title_status,transmission,type,state,lat,long
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,nissan,gas,32226.0,clean,other,sedan,wy,33.786500,-84.445400
426876,wyoming,30590,2020.0,volvo,gas,12029.0,clean,other,sedan,wy,33.786500,-84.445400
426877,wyoming,34990,2020.0,cadillac,diesel,4174.0,clean,other,hatchback,wy,33.779214,-84.411811
426878,wyoming,28990,2018.0,lexus,gas,30112.0,clean,other,sedan,wy,33.786500,-84.445400


In [110]:
data_v2_price = data_v2['price'].copy()

In [111]:
data_v2_price

0          6000
1         11900
2         21000
3          1500
4          4900
          ...  
426875    23590
426876    30590
426877    34990
426878    28990
426879    30590
Name: price, Length: 426880, dtype: int64

In [112]:
data_v2 = data_v2.drop('price', axis = 1)

In [113]:
data_v2

,region,year,manufacturer,fuel,odometer,title_status,transmission,type,state,lat,long
0,prescott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,2019.0,nissan,gas,32226.0,clean,other,sedan,wy,33.786500,-84.445400
426876,wyoming,2020.0,volvo,gas,12029.0,clean,other,sedan,wy,33.786500,-84.445400
426877,wyoming,2020.0,cadillac,diesel,4174.0,clean,other,hatchback,wy,33.779214,-84.411811
426878,wyoming,2018.0,lexus,gas,30112.0,clean,other,sedan,wy,33.786500,-84.445400


In [114]:
data_v2_cat = data_v2.drop(['year', 'odometer', 'lat', 'long'], axis = 1)
data_v2_cat

,region,manufacturer,fuel,title_status,transmission,type,state
0,prescott,NaN,NaN,NaN,NaN,NaN,az
1,fayetteville,NaN,NaN,NaN,NaN,NaN,ar
2,florida keys,NaN,NaN,NaN,NaN,NaN,fl
3,worcester / central MA,NaN,NaN,NaN,NaN,NaN,ma
4,greensboro,NaN,NaN,NaN,NaN,NaN,nc
...,...,...,...,...,...,...,...
426875,wyoming,nissan,gas,clean,other,sedan,wy
426876,wyoming,volvo,gas,clean,other,sedan,wy
426877,wyoming,cadillac,diesel,clean,other,hatchback,wy
426878,wyoming,lexus,gas,clean,other,sedan,wy


In [131]:
num_attribs = ['year', 'odometer', 'lat', 'long']
num_attribs

['year', 'odometer', 'lat', 'long']

In [132]:
data_v2_num = data_v2[num_attribs].copy()

In [133]:
data_v2_num

,year,odometer,lat,long
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
426875,2019.0,32226.0,33.786500,-84.445400
426876,2020.0,12029.0,33.786500,-84.445400
426877,2020.0,4174.0,33.779214,-84.411811
426878,2018.0,30112.0,33.786500,-84.445400


In [134]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [135]:

cat_attribs = ['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'type', 'state']

num_pipeline = Pipeline([
              ('impute', SimpleImputer(strategy = 'mean')),
              ('std_scaler', StandardScaler()),
])
cat_pipeline = Pipeline([
              ('selector', DataFrameSelector(cat_attribs)),
              ('cat_encoder', OneHotEncoder(categories=cat_attribs)),
])

In [136]:
cat_attribs

['region',
 'manufacturer',
 'fuel',
 'title_status',
 'transmission',
 'type',
 'state']

In [137]:
(num_attribs,
 cat_attribs)

(['year', 'odometer', 'lat', 'long'],
 ['region',
  'manufacturer',
  'fuel',
  'title_status',
  'transmission',
  'type',
  'state'])

In [138]:
oHe = OneHotEncoder()

In [139]:
full_pipeline = ColumnTransformer([
                                               ('num_pipeline', num_pipeline, num_attribs),
                                               ('cat_pipeline', OneHotEncoder(), cat_attribs),

                                               
])

In [140]:
data_v2

,region,year,manufacturer,fuel,odometer,title_status,transmission,type,state,lat,long
0,prescott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,2019.0,nissan,gas,32226.0,clean,other,sedan,wy,33.786500,-84.445400
426876,wyoming,2020.0,volvo,gas,12029.0,clean,other,sedan,wy,33.786500,-84.445400
426877,wyoming,2020.0,cadillac,diesel,4174.0,clean,other,hatchback,wy,33.779214,-84.411811
426878,wyoming,2018.0,lexus,gas,30112.0,clean,other,sedan,wy,33.786500,-84.445400


In [141]:
dataset_prepared = full_pipeline.fit_transform(data_v2)

In [142]:
dataset_prepared

<426880x533 sparse matrix of type '<class 'numpy.float64'>'
	with 4684731 stored elements in Compressed Sparse Row format>

In [143]:
import scipy.sparse

In [144]:
data_v2_prepared = pd.DataFrame.sparse.from_spmatrix(dataset_prepared)

In [145]:
data_v2_prepared

,0,1,2,3,4,5,6,7,8,9,...,523,524,525,526,527,528,529,530,531,532
0,-2.408936e-14,0.000000,0.000000,2.339361e-15,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-2.408936e-14,0.000000,0.000000,2.339361e-15,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-2.408936e-14,0.000000,0.000000,2.339361e-15,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-2.408936e-14,0.000000,0.000000,2.339361e-15,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-2.408936e-14,0.000000,0.000000,2.339361e-15,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,8.226515e-01,-0.309327,-0.812111,5.653637e-01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
426876,9.285977e-01,-0.404248,-0.812111,5.653637e-01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
426877,9.285977e-01,-0.441165,-0.813368,5.672068e-01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
426878,7.167054e-01,-0.319262,-0.812111,5.653637e-01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [146]:
data_v2_prepared.isna().sum().sum()

0

In [152]:
from sklearn.linear_model import LinearRegression
lin_reg_v3 = LinearRegression()

In [157]:
scaler = StandardScaler()

In [153]:
data_v2_price

0          6000
1         11900
2         21000
3          1500
4          4900
          ...  
426875    23590
426876    30590
426877    34990
426878    28990
426879    30590
Name: price, Length: 426880, dtype: int64

In [159]:
data_v2_price_prepared = num_pipeline.fit_transform(data_v2_price)


ValueError: Expected 2D array, got 1D array instead:
array=[ 6000. 11900. 21000. ... 34990. 28990. 30590.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [149]:
lin_reg_v2.fit(data_v2_prepared, data_v2_price_prepared)

LinearRegression()

In [151]:
some_data = data_v2.iloc[:5]
some_labels = data_v2_price.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg_v2.predict(some_data_prepared))

print("Labels:", list(some_labels))


Predictions: [-42258.27599039 -53126.53375231  43842.08447351 -30089.10113984
 -49700.15506704]
Labels: [6000, 11900, 21000, 1500, 4900]


In [ ]:
# Well that's not good :(